In [1]:
import sys
import os
import time
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sys.path.append("../0.mc_utils/")

In [3]:
from detector.yolov4_config import YOLO_CFG
from detector.yolov4_wrapper import YoloV4Detector

from common.one_eurio_filter import OneEuroFilter
from common.unity_visualizer import draw_kp2d_to_image
from common.tools_cv import draw_kp

from syp_hmr_e2e.x2ds_config import X2DS_CPN50_CFG
from syp_hmr_e2e.x2ds_cpn50_wrapper import X2dsCPN50Wrapper

>>>syp_hmr_e2e path:D:\XiaHaiPeng\20200520-3dat-xiga-v2.1\0.mc_utils\syp_hmr_e2e


In [4]:
WIDTH=56
HEIGHT=56
device= "cuda:0"

x2ds_filter = OneEuroFilter(min_cutoff=0.05, beta=0.03,d_cutoff=0.02)

In [5]:
cfg = YOLO_CFG.clone()
# cfg.confidence_threshold=0.8
cfg.device = device
cfg.framesize = 320
print(cfg)
human_detector = YoloV4Detector()

classes: [0]
confidence_threshold: 0.5
device: cuda:0
framesize: 320
iou_threshold: 0.6
model: ../12.models/yolov5/yolov5s.pt
>>>YoloV4 loaded D:\XiaHaiPeng\20200520-3dat-xiga-v2.1\12.models\yolov5\yolov5s.pt making a random inference.


In [6]:
################
## exp-resnet ##
################
x2ds_cfg = X2DS_CPN50_CFG.clone()
x2ds_cfg.device = device
x2ds_cfg.basic.xyxy_conf_thresh = 0.5
x2ds_cfg.basic.crop_scale = (1.0,1.1)
x2ds_cfg.conf_reader.sigma = 3.0
x2ds_cfg.model = "../12.models/hmr_e2e/20200904-cpn50-224x224j20-climb-0.5507786451159297.pth"
# x2ds_cfg.model = "../12.models/hmr_e2e/20200817-cpn50-224x224j20-2.pth"
x2ds_cfg.network.hmap_channels = 20
print(x2ds_cfg)
x2ds_resnet50 = X2dsCPN50Wrapper(x2ds_cfg)

basic:
  crop_scale: (1.0, 1.1)
  down_scale: 4.0
  hmap_size: (56, 56)
  out_size: (224, 224)
  xyxy_conf_thresh: 0.5
conf_reader:
  conf_threashold: 0.1
  sigma: 3.0
device: cuda:0
model: ../12.models/hmr_e2e/20200904-cpn50-224x224j20-climb-0.5507786451159297.pth
network:
  block_channels: 256
  hmap_channels: 20
>>>>Loaded model:../12.models/hmr_e2e/20200904-cpn50-224x224j20-climb-0.5507786451159297.pth making a random inference.


In [7]:
# video_fname = "E:/test_videos/climbing/2020-0831/climbing.mp4"
# video_fname = "E:/test_videos/climbing/0828攀岩测试/0828_手机拍摄/VID_20200827_153433.mp4"
# video_fname = "E:/test_videos/climbing/0828攀岩测试/0828_摄像头拍摄/PY/20200827_153923.mp4"
# video_fname = "E:/test_videos/climbing/0828攀岩测试/0828_摄像头拍摄/PY/20200827_153806.mp4"
# video_fname = "E:/test_videos/climbing/0828攀岩测试/0828_摄像头拍摄/PY/20200827_165133.mp4"
# video_fname = "E:/test_videos/climbing/0828攀岩测试/0828_手机拍摄/VID_20200827_161932.mp4"
#video_fname = "E:/test_videos/climbing/make_train_set/20200827_155822.mp4"
video_fname=700

cap = cv2.VideoCapture(video_fname)

# cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,720)


window_name = "yolov5-dsnt2d"
cv2.namedWindow(window_name)
frame_count = 0
framewidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)/2)
frameheight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)/2)

#
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = cap.get(cv2.CAP_PROP_FPS)
size = (framewidth*2, frameheight*2)

# 保存视频
# out = cv2.VideoWriter('E:/test_videos/climbing/0828攀岩测试/out_phone/outVideo_1.mp4', fourcc, fps, size)
# out = cv2.VideoWriter('E:/test_videos/climbing/0828攀岩测试/out_cam/outVideo_1.mp4', fourcc, fps, size)
# out = cv2.VideoWriter('E:/test_videos/climbing/0828攀岩测试/out_cam/outVideo_153806.mp4', fourcc, fps, size)
out = cv2.VideoWriter('E:/test_videos/climbing/0828攀岩测试/out_cam/outVideo_155822.mp4', fourcc, fps, size)
# out = cv2.VideoWriter('E:/test_videos/outVideo_retrain.mp4', fourcc, fps, size)

im_number = 0
t = time.time()
while True:

    is_valid,frame = cap.read()
    if not is_valid:
        break
    frame_count += 1
    #frame = cv2.resize(frame,(framewidth,frameheight))
    ##step1 preprocess image
    raw_image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) #HxWxC
    
        
    ##step2.human detector
    det_ret = human_detector(raw_image)
    if det_ret is not None:
        batch_xyxy = det_ret["xyxy"]
        batch_conf = det_ret["conf"]
        ##step2.prepare to 2D image
        for i in range(batch_xyxy.shape[0]):
            raw_xyxy = batch_xyxy[i]
            cv2.rectangle(frame,(raw_xyxy[0],raw_xyxy[1]),(raw_xyxy[2],raw_xyxy[3]),(0,255,0),2)
            cv2.putText(frame, 
                        "P:{:.3}".format(batch_conf[i]), (raw_xyxy[0],raw_xyxy[1]), 
                         cv2.FONT_HERSHEY_SIMPLEX, 
                         0.6, 
                         (0,255,0), 
                         1,
                         cv2.LINE_AA)
    
        ##step3.2D pose to heatmap
        pred = x2ds_resnet50(raw_image,batch_xyxy,batch_conf)
        if pred is not None:
            pred_x2ds = pred[...,:2]
            pred_conf = pred[...,-1:]
            conf_x2ds = pred_x2ds*(pred_conf>0.1)

            ##step6.visualize x2ds
            frame = draw_kp(frame,conf_x2ds,fmt="mc16",color=(0,255,0))
            for i in range(len(pred_x2ds)):
    #             draw_kp2d_to_image(pred_x2ds[i],frame,radius=4,font_scale=0.6,color=(0,255,0))
                draw_kp2d_to_image(conf_x2ds[i],frame,radius=4,font_scale=0.6,color=(255,0,0))

    # 保存视频
#     print(frame_count,frame.shape,end="\r")
#     out.write(frame)
    fps = 1/(time.time()-t)
    cv2.putText(frame, ('fps:%.2f'%fps), (10,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
    cv2.imshow(window_name,frame)
    t=time.time()
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [9]:
cv2.destroyAllWindows()
cap.release()